In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import UndefinedMetricWarning
import os
import time

import warnings
# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Load data
data_dir = 'sequences'
data = torch.load(os.path.join(data_dir, 'data_balanced.pt'))
X = data['X']
y = data['y']

# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Create DataLoaders
batch_size = 16
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory to save the best models
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Define training loop
def train(model, train_loader, val_loader, epochs=10, patience=5, model_class=None, description=None):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validation
        val_loss = validate(model, val_loader, criterion)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}')
        
        if description:
            model_path = os.path.join(models_dir, f'{description}_{name.replace(" ", "_").lower()}.pth')
        else:
            model_path = os.path.join(models_dir, f'{name.replace(" ", "_").lower()}.pth')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model, model_path)
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print('Early stopping!')
            model = torch.load(model_path)  # Load the entire model
            return

def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1).float())
            total_loss += loss.item()
    return total_loss / len(val_loader)


def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            preds = torch.sigmoid(outputs) > 0.5
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print('Accuracy:', accuracy)
    print('Classification Report:')
    print(classification_report(all_labels, all_preds))
    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, all_preds))
    return accuracy

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, x):
        raise NotImplementedError

class FlattenModel(BaseModel):
    def forward(self, x):
        return self._forward(x.view(x.size(0), -1))

    def _forward(self, x):
        raise NotImplementedError

class LogisticRegression(FlattenModel):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def _forward(self, x):
        return self.linear(x)

class MLP(FlattenModel):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    def _forward(self, x):
        return self.layers(x)

class LSTMModel(BaseModel):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1])

class BidirectionalLSTMModel(BaseModel):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BidirectionalLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)  # *2 because it's bidirectional
    
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        x = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
        return self.fc(x)


# Update the models dictionary
input_dim = 64 * 8  # Assuming your input shape is (batch_size, 64, 8)
models = {
    'Logistic Regression': LogisticRegression(input_dim),
    'MLP': MLP(input_dim),
    'LSTM': LSTMModel(input_size=8, hidden_size=128, num_layers=1),
    'Bidirectional LSTM': BidirectionalLSTMModel(input_size=8, hidden_size=128, num_layers=1),
}


train_description = "v3"

# Train and evaluate each model
results = {}
for name, model in models.items():
    print(f"\nTraining {name}")
    start_time = time.time()
    train(model, train_loader, val_loader, epochs=70, patience=5, model_class=type(model), description=train_description)
    train_time = time.time() - start_time

    print(f"\nEvaluating {name}")
    accuracy = evaluate(model, test_loader)
    
    results[name] = {'accuracy': accuracy, 'train_time': train_time}

# Print summary of results
print("\nSummary of Results:")
for name, result in results.items():
    print(f"{name}: Accuracy = {result['accuracy']:.4f}, Training Time = {result['train_time']:.2f} seconds")


Training Logistic Regression
Epoch 10, Train Loss: 0.5820, Val Loss: 0.6100
Epoch 20, Train Loss: 0.5060, Val Loss: 0.5327
Epoch 30, Train Loss: 0.4579, Val Loss: 0.4773
Epoch 40, Train Loss: 0.4223, Val Loss: 0.4530
Epoch 50, Train Loss: 0.3991, Val Loss: 0.4354
Epoch 60, Train Loss: 0.3785, Val Loss: 0.4146
Epoch 70, Train Loss: 0.3665, Val Loss: 0.4096

Evaluating Logistic Regression
Accuracy: 0.9142857142857143
Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93        20
         1.0       1.00      0.80      0.89        15

    accuracy                           0.91        35
   macro avg       0.93      0.90      0.91        35
weighted avg       0.93      0.91      0.91        35

Confusion Matrix:
[[20  0]
 [ 3 12]]

Training MLP
Epoch 10, Train Loss: 0.3853, Val Loss: 0.4438
Epoch 20, Train Loss: 0.2806, Val Loss: 0.3605
Epoch 30, Train Loss: 0.2507, Val Loss: 0.2957
Epoch 40, Train Loss: 0.2228, Val Loss

In [36]:
import os

# Get all data files in the sequences directory
data_files = [file for file in os.listdir('sequences') if file.startswith('data_balanced_seq')]
print(data_files)

# Train and evaluate each model on each data file
results = {}
for data_file in data_files:
    # Extract sequence length and overlap from file name
    parts = data_file.split('_')
    seq_len = int(parts[4])
    overlap = int(parts[-1].split('.')[0])
        
    # Load data
    data = torch.load(os.path.join('sequences', data_file))
    X = data['X']
    y = data['y']
    
    # Split data into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)
    
    # Create DataLoaders
    batch_size = 16
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Update the models dictionary
    input_dim = seq_len * 8  # Assuming your input shape is (batch_size, seq_len, 8)
    models = {
        'Logistic Regression': LogisticRegression(input_dim),
        'MLP': MLP(input_dim),
        'LSTM': LSTMModel(input_size=8, hidden_size=128, num_layers=1),
        'Bidirectional LSTM': BidirectionalLSTMModel(input_size=8, hidden_size=128, num_layers=1),
    }
    
    train_description = f"seqlen{seq_len}overlap{overlap}"
    
    # Train and evaluate each model
    for name, model in models.items():
        print(f"\nTraining {name} on {data_file}")
        start_time = time.time()
        train(model, train_loader, val_loader, epochs=70, patience=5, model_class=type(model), description=train_description)
        train_time = time.time() - start_time
        
        print(f"\nEvaluating {name} on {data_file}")
        accuracy = evaluate(model, test_loader)
        
        results[f"{name} on {data_file}"] = {'accuracy': accuracy, 'train_time': train_time}

# Print summary of results
print("\nSummary of Results:")
for name, result in results.items():
    print(f"{name}: Accuracy = {result['accuracy']:.4f}, Training Time = {result['train_time']:.2f} seconds")

['data_balanced_seq_len_128_overlap_16.pt', 'data_balanced_seq_len_128_overlap_32.pt', 'data_balanced_seq_len_128_overlap_64.pt', 'data_balanced_seq_len_32_overlap_16.pt', 'data_balanced_seq_len_32_overlap_32.pt', 'data_balanced_seq_len_32_overlap_64.pt', 'data_balanced_seq_len_64_overlap_16.pt', 'data_balanced_seq_len_64_overlap_32.pt', 'data_balanced_seq_len_64_overlap_64.pt']

Training Logistic Regression on data_balanced_seq_len_128_overlap_16.pt
Epoch 10, Train Loss: 0.5092, Val Loss: 0.5571
Epoch 20, Train Loss: 0.4164, Val Loss: 0.4708
Epoch 30, Train Loss: 0.3719, Val Loss: 0.4393
Epoch 40, Train Loss: 0.3395, Val Loss: 0.4209
Epoch 50, Train Loss: 0.3198, Val Loss: 0.4082
Epoch 60, Train Loss: 0.2958, Val Loss: 0.3990
Epoch 70, Train Loss: 0.2812, Val Loss: 0.3980

Evaluating Logistic Regression on data_balanced_seq_len_128_overlap_16.pt
Accuracy: 0.82
Classification Report:
              precision    recall  f1-score   support

         0.0       0.71      0.83      0.77     